In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
location = 'kaggle/'
events = pd.DataFrame.from_csv(location+"user_activity.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"user_activity_test.csv",index_col=None)

In [4]:
events

,user_id,action,step_id,time,step_type,step_cost
0,5920,discovered,2681,1403165936,text,0
1,5920,passed,2681,1403165936,text,0
2,5920,viewed,2681,1403165936,text,0
3,5920,passed,2682,1403165946,text,0
4,5920,viewed,2682,1403165946,text,0
5,5920,discovered,2682,1403165946,text,0
6,5920,passed,2687,1403165994,text,0
7,5920,discovered,2687,1403165994,text,0
8,5920,viewed,2687,1403165994,text,0
9,5920,passed,2688,1403166007,text,0


In [16]:
sc = events.groupby('user_id')

In [21]:
def getX(ev):
    sc  = ev.groupby('user_id')
    features = sc.sum().step_cost.values.reshape(-1, 1)
    features = np.concatenate((features, sc.sum().time.reshape((-1, 1))), axis=1)
    features = np.concatenate((features, sc.mean().time.reshape((-1, 1))), axis=1)
    features = np.concatenate((features, sc.mean().step_cost.reshape((-1, 1))), axis=1)
    features = np.concatenate((features, sc.size().reshape((-1, 1))), axis=1)
    #features = np.concatenate((features, ), axis=1)
    print(features)
    return (sc.sum().index, features)

array([[254],
       [  9],
       [ 31],
       ..., 
       [  7],
       [  1],
       [  1]])

In [22]:
ind, X = getX(events)

[[  9.97000000e+02   2.85221266e+12   1.46492689e+09   5.12069851e-01
    1.94700000e+03]
 [  0.00000000e+00   4.22767219e+10   1.45781800e+09   0.00000000e+00
    2.90000000e+01]
 [  5.00000000e+01   1.87801385e+11   1.45582469e+09   3.87596899e-01
    1.29000000e+02]
 ..., 
 [  0.00000000e+00   2.96252875e+10   1.41072797e+09   0.00000000e+00
    2.10000000e+01]
 [  0.00000000e+00   5.64291101e+09   1.41072775e+09   0.00000000e+00
    4.00000000e+00]
 [  0.00000000e+00   4.23218099e+09   1.41072700e+09   0.00000000e+00
    3.00000000e+00]]


In [24]:
a = sc.apply(lambda x: x[x.action == 'passed']).groupby('user_id').size().reshape((-1, 1))

In [31]:
a = np.resize(a, (16625, 1))

In [32]:
a.shape

(16625, 1)

In [34]:
X =np.concatenate((X, a), axis=1)

In [35]:
ind.shape

(16625,)

In [36]:
ind_test, X_test = getX(events_test)

[[  0.00000000e+00   4.39553390e+09   1.46517797e+09   0.00000000e+00
    3.00000000e+00]
 [  1.10000000e+01   8.93752355e+10   1.46516780e+09   1.80327869e-01
    6.10000000e+01]
 [  6.00000000e+00   2.47799033e+10   1.45764137e+09   3.52941176e-01
    1.70000000e+01]
 ..., 
 [  0.00000000e+00   2.11610038e+10   1.41073359e+09   0.00000000e+00
    1.50000000e+01]
 [  0.00000000e+00   4.23219681e+09   1.41073227e+09   0.00000000e+00
    3.00000000e+00]
 [  0.00000000e+00   2.96252877e+10   1.41072799e+09   0.00000000e+00
    2.10000000e+01]]


In [42]:
sc = events_test.groupby('user_id')
a = sc.apply(lambda x: x[x.action == 'passed']).groupby('user_id').size().reshape((-1, 1))

In [43]:
a.shape

(4180, 1)

In [44]:
a = np.resize(a, (4255, 1))
X_test =np.concatenate((X_test, a), axis=1)

In [45]:
X_test.shape

(4255, 6)

In [46]:
Y = targets.passed.values


In [47]:
Y.shape

(16625,)

In [81]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
#model = DummyClassifier( constant=0)
model =  DecisionTreeClassifier()

In [82]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
#Поделим нашу выборку на train и validation
Xtr,Xval,Ytr,Yval = train_test_split(X,Y,test_size=0.1,random_state=128)

In [83]:
import tensorflow.contrib.learn.python.learn as learn
feature_columns = learn.infer_real_valued_columns_from_input(Xtr)
classifier = learn.DNNClassifier(hidden_units=[10], n_classes=2, feature_columns=feature_columns)
classifier.fit(Xtr, Ytr, steps=200, batch_size=32)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/tensorflow/python/ops/array_ops.py:1750: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  result_shape.insert(dim, 1)


KeyboardInterrupt: 

In [84]:
#Обучим модель
model.fit(Xtr, Ytr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [85]:
#Оценим метрику на validation
print(f1_score(Yval, model.predict(Xval)))

0.256


In [62]:
classifier.predict(Xval).mean()

0.0

In [54]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

In [55]:
#Предскажем результат и запишем его в .csv
ans = model.predict(X_test)
result = np.concatenate((np.asarray(ind_test, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, 'submissions/rf-5feature')